In [84]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]="Your_api_key"

In [8]:
! pip install -q langchain-huggingface langchain-core langchain_community requests duckduckgo-search
! pip install -U ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


### All important imports

In [67]:
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from langchain_core.tools import tool
import requests
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [68]:
#taking user query
search_topic=input("Enter the topic you want to search : ")

Enter the topic you want to search : weather today in india?


### create search tool for latest info

In [69]:
# search internet for requirements
search_tool=DuckDuckGoSearchRun()
result=search_tool.invoke(search_topic)


In [70]:
#checking if result is stored or not tool is working?
result

'India Weather Today : India braces for wet week ahead; rains disrupt life in Tripura, Bengaluru ... The weather agency, as per a report in The Indian ... India Weather Today : Light to moderate rainfall, thunderstorms, and strong winds likely in parts of India this week According to experts quoted by news agency PTI , temperatures may even scale to 47 degrees Celsius in parts of northwest India . Today , in Padmanabham, expect scattered showers to bring rain on and off throughout the day. ... filled and occasionally cloudy weather is forecasted. Weather forecast and temperature for today - Saturday, 25. ... At the moment, in New Delhi, the weather is bright and sunny .'

# **Creating tool for weather info**

In [71]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key={your_API_key}&query={city}'

  response = requests.get(url)

  return response.json()

### LLM initilizing

In [72]:
model = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)
llm=ChatHuggingFace(llm=model)

In [73]:
llm.invoke('hi') #checking if model is working or not

AIMessage(content='Hi! 👋  What can I do for you today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 10, 'total_tokens': 25}, 'model_name': 'google/gemma-2-2b-it', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--9884f0b7-adb5-46ec-9f5b-30b7a0486ac6-0', usage_metadata={'input_tokens': 10, 'output_tokens': 15, 'total_tokens': 25})

### pulling react prompt from langchain hub
**standard  ReAct agent prompt**

In [74]:
prompt=hub.pull("hwchase17/react")

# Manually creating react agent

In [75]:
from re import VERBOSE
agent=create_react_agent(
    llm=llm,
    tools=[search_tool,get_weather_data],
    prompt=prompt
)

## Wrape it with agent executor

In [76]:
agent_executor=AgentExecutor(
    agent=agent,
    tools=[search_tool,get_weather_data],
    verbose=True
)


In [77]:
# creating query so that user can enter it
query=input("Enter your query : ")

Enter your query : i want to visit mumbai to hongkong by flight can you tell me price of ticket and weather  as well


**Invoking agent**

In [78]:
response=agent_executor.invoke({"input":{query}})
print(response)



> Entering new AgentExecutor chain...
Thought: I need to get information about flight prices and weather for Mumbai and Hong Kong.
Action: duckduckgo_search
Action Input: flight price mumbai hong kongCombine your flight from Mumbai to Hong Kong with hotels and car hire. Once you've secured your plane ticket we will also help you discover the best prices and exclusive rates for hotels in Hong Kong or close by. Average price of one-stop return flights from Mumbai to Hong Kong , with the same stopover airport on both legs, clicked by momondo users over the past 12 months.A good Cathay Pacific flight deal for Mumbai to Hong Kong starts from ₹ 399 round-trip, up to an average price of ₹ 522. The price you pay for your flight from Mumbai to Hong Kong may vary depending on when you book.The cheapest flight overall is ₹ 23,618 while the most popular route, ( Mumbai Chhatrapati Shivaji Intl - Hong Kong Intl) is currently priced at ₹ 24,602. Last-minute promo tickets for the route Mumbai - Hon

In [79]:
ans=response['output']

## Creating prompt for llm to summarise the output and parser as well

In [80]:
do = PromptTemplate(
    input_variables=["ans"],
    template="""
You are a polite and professional female receptionist at a travel agency.
Your task is to take the AI agent's answer and rephrase it in a warm, formal, and professional manner.

Guidelines for your reply:
1. Begin with a courteous greeting.
2. Rephrase the AI's answer clearly and politely.
3. If the answer asks for details, phrase it as thoughtful and professional questions.
4. Add helpful travel reminders when appropriate (documents, tickets, ID, travel insurance, essentials).
5. Keep it concise, professional, and friendly.

Here is the AI agent’s answer that you must rephrase:
"{ans}"
"""
)

In [81]:
parser=StrOutputParser()

now chain to invoke it


In [82]:
chain= do|llm|parser

# **invoking chain**

In [83]:
# Run it
output = chain.invoke({"ans": response['output']})
print(output)

Hello, welcome!  It sounds like you're planning a trip to Mumbai and Hong Kong.  I'm happy to share some preliminary information on your flights. The estimated prices for a round trip from Mumbai to Hong Kong are quite varied. 

For Mumbai, the summer months we live with temperatures around 30 degrees Celcius with a degree of haze.  In Hong Kong, expect 33 degrees Celsius, a little more warmth.  Both these cities are tropical, so please remember to pack accordingly.

To ensure a smooth travel experience, it's a good idea to remember the essential travel documents, including your passport and both confirmed tickets, particularly if traveling during peak seasons. It's essential to also ensure you have travel insurance and any required visas. Enjoy your trip! 

